In [2]:
import numpy as np
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random
from jax.nn import softmax, one_hot

In [3]:
from datasets import load_dataset


dataset = load_dataset("wikitext", "wikitext-2-raw-v1")
train_data = dataset['train']
val_data = dataset['validation']
test_data = dataset['test']

In [1]:
corpus = ""
for i in range(len(train_data["text"])):
  corpus= corpus +"."+ train_data[i]["text"]
  # print(i)
# print(corpus)

NameError: name 'train_data' is not defined

In [2]:
# corpus

In [7]:
corpus = corpus.replace('=', '').replace('@', '').replace('.', '').replace(',', '').replace('(', '').replace(')', '').replace('"', '')
corpus = corpus.split('\n')
corpus = [i for i in corpus if len(i.split(' ')) > 5]

In [3]:
# corpus

In [8]:
def tokenize_corpus(corpus):
    tokens = [x.split() for x in corpus]
    return tokens

In [9]:
def create_vocabulary(tokenized_corpus):
    vocab = {}
    vocab['<PAD>'] = 0
    i = 1
    for sentence in tokenized_corpus:
        for word in sentence:
            if word in vocab:
                continue
            vocab[word] = i
            i += 1
            
    return vocab

In [16]:
def generate_train_data(tokenized_corpus, vocab, window_size):
    X = []
    y = []
    k=0
    for phrase in tokenized_corpus:
        for i in range(len(phrase)):
            target = i
            context = []
            for j in range(-window_size, window_size + 1):
                if j == 0:
                    continue
                if i + j < 0 or i + j>=len(phrase):
                    context.append('<PAD>')
                else:
                    context.append(phrase[i + j])
        
            X.append([vocab[k] for k in context])
            y.append(vocab[phrase[target]])
    return jnp.array(X), jnp.array(y)

In [17]:
window_size = 2

In [18]:
tokenized_corpus = tokenize_corpus(corpus)
vocab = create_vocabulary(tokenized_corpus)
len(tokenized_corpus)

23318

In [1]:
X, y = generate_train_data(tokenized_corpus[:1000], vocab, window_size)

NameError: name 'generate_train_data' is not defined

In [4]:
# vocab

In [5]:
# tokenized_corpus

In [20]:
len(X)

1789310

In [21]:
y

Array([   1,    2,    3, ...,  145, 2454, 2317], dtype=int32)

In [22]:
class Word2VecCBOW():
    def __init__(self, window_size, embed_dim, vocab_size, random_state):
        # Defines the key to be used for the random creation of the weights
        self.key = random.PRNGKey(random_state)
        self.vocab_size = vocab_size
        self.linear = self._create_random_matrix(vocab_size, embed_dim)
        self.soft = self._create_random_matrix(embed_dim, vocab_size)
        
        # Vectorizes the predict method
        self.predict = vmap(self._predict, in_axes=(None, 0))

    def train(self, X, y, num_epochs, batch_size):
        y = one_hot(y, self.vocab_size)
        X = one_hot(X, self.vocab_size)
        params = [self.linear, self.soft]
        for epoch in range(num_epochs):
            print(f'Epoch: {epoch}')
            for X_batch, y_batch in self.generate_batches(X, y, batch_size):
                params = self.update(params, X_batch, y_batch)
            print(f'Loss: {float(self.l.primal)}')
        self.linear = params[0]
        self.soft = params[1]

    def _predict(self, params, X):
        activations = []
        for x in X:
            activations.append(jnp.dot(x, params[0]))
    # Averages the activations
        activation = jnp.mean(jnp.array(activations), axis=0)
        logits = jnp.dot(activation, params[1])
        result = softmax(logits)
        
        return result

    def _create_random_matrix(self, window_size, embed_dim):
        w_key = random.split(self.key, num=1)
        
        return 0.2 * random.normal(self.key, (window_size, embed_dim))

    def loss(self, params, X, y):
        preds = self.predict(params, X)
        l = -jnp.mean(preds * y)
        self.l = l
        return l

    def update(self, params, X, y, step_size=0.02):
        grads = grad(self.loss)(params, X, y)
        return [params[0] - step_size * grads[0],
                params[1] - step_size * grads[1]]

    def get_embedding(self):
        return self.linear

    def generate_batches(self, X, y, batch_size):
        for index, offset in enumerate(range(0, len(y), batch_size)):
            yield X[offset: offset + batch_size], y[offset: offset + batch_size]

In [23]:
w2v = Word2VecCBOW(2, 32, len(vocab), 42)
w2v.train(X, y, 2, 1000)
w2v.get_embedding()